In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import datetime

In [3]:
import sys

In [4]:
sys.path.append('../')

from lib import lib

In [5]:
li_currency = ["USDJPY","EURJPY","EURUSD","GBDJPY","CADJPY","CHFJPY","GBDUSD","USDCHF","SEKJPY","NOKJPY",
"EURGBP","USDCAD","TRYJPY","ZARJPY","MXNJPY","TRYUSD","EURTRY","RUBJPY","AUDJPY","NZDJPY",
"AUDUSD","NZDUSD","EURAUD","GBPAUD","AUDCAD","EURNZD","AUDNZD","CNYJPY","HKDJPY","SGDJPY"]

In [22]:
li_tweet = ["USDJPY","EURUSD","GBPUSD","AUDUSD"]

In [8]:
li_time = [3,6,12,24]

In [9]:
tm = datetime.datetime.now()
tm = tm - datetime.timedelta(minutes=tm.minute % 10,seconds=tm.second,microseconds=tm.microsecond)
tm =tm.strftime('%Y-%m-%d-%H-%M-%S')
li_tm = list(map(int,tm.split("-")))
li_tm

[2021, 4, 22, 13, 30, 0]

In [10]:
date = datetime.date(li_tm[0],li_tm[1],li_tm[2])
dayweek = date.strftime('%A')
boolDay = False
if dayweek == "Sunday":
    boolDay = False
elif dayweek == "Saturday" and tm[3] > 6:
    boolDay = False
elif dayweek == "Monday" and tm[3] < 6:
    boolDay = False
else:
    boolDay = True
boolDay

True

In [11]:
PERIOD2 = 6 # 変化の間隔

In [13]:
import pickle

In [23]:
for PERIOD in li_time:
    data = lib.makeData(li_currency).values
    diffs = lib.diffData(data,PERIOD2)
    data = data[PERIOD2:]
    data = np.concatenate([data,diffs],1)[max(PERIOD-PERIOD2,0):]
    data = [data[-1]]
    print(data)
    for i in li_tweet:
        filename = "../models/model/" + i + "/r" + str(PERIOD) + "_0.sav"
        model = pickle.load(open(filename, 'rb'))
        result = model.predict(data)
        print(i, PERIOD, result[0])

[array([2, 2, 1, ..., 0, 0, 0], dtype=int64)]
USDJPY 3 0.0035508567630733417
EURUSD 3 0.00014920989883258727
GBPUSD 3 5.746959462668173e-06
AUDUSD 3 0.00010150964383915761
[array([2, 2, 1, ..., 0, 0, 0], dtype=int64)]
USDJPY 6 0.0009748497902709337
EURUSD 6 0.00010477299047667048
GBPUSD 6 0.00010832412235670202
AUDUSD 6 5.906895768490624e-05
[array([2, 2, 1, ..., 0, 0, 0], dtype=int64)]
USDJPY 12 -0.005965110738848972
EURUSD 12 0.0008155177466765158
GBPUSD 12 0.0005426180710734535
AUDUSD 12 0.00012720676137814994
[array([2, 2, 1, ..., 0, 0, 0], dtype=int64)]
USDJPY 24 -0.08479766800289883
EURUSD 24 0.0006612706018393819
GBPUSD 24 0.0009325735792036783
AUDUSD 24 0.00018669704959981876


In [23]:
ans1 = 0 # 0,1,-1
ans2 = 0 # 0,2,-2
ans3 = 0 # 0,1,-1
ans4 = 0 # 0,2,-2
ans5 = 0 # 0,1,-1, 2,-2
ansli = [ans1,ans2,ans3,ans4,ans5]

In [24]:
def textModel(x):
    if x == 0:
        return "変化なし"
    elif x == 1:
        return "弱い買い"
    elif x == 2:
        return "強い買い"
    elif x == -1:
        return "弱い売り"
    elif x == -1:
        return "強い売り"
textli = list(map(textModel,ansli))

In [29]:
timeli = [" 30分後","1時間後","2時間後","4時間後","8時間後"]

In [33]:
import twitter

In [ ]:
with open( "./twitterpass.txt" ,"a",newline="") as f:
    li = f.read().splitlines()
    myConsumerKey = li[0]
    myConsumerSecret = li[1]
    myToken = li[2]
    myTokenSecret = li[3]

In [ ]:
auth = twitter.OAuth(
    consumer_key=myConsumerKey,
    consumer_secret=myConsumerSecret,
    token=myToken,
    token_secret=myTokenSecret)

In [37]:
t = twitter.Twitter(auth=auth)

In [41]:
def makeText(c, t, x):
    txt = c + "の価格予測\n"
    for i in range(5):
        txt += t[i] + "   " + x[i] + "\n"
    return txt

In [ ]:
status=makeText("USD/JPY",timeli,textli) #投稿するツイート
t.statuses.update(status=status) #Twitterに投稿